In [1]:
import pandas as pd
import datetime as dt
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Getting 538 data

tables = pd.read_html(f'https://projects.fivethirtyeight.com/{dt.date.today().year}-nfl-predictions/games/?ex_cid=rrpromo')
# Instantiating final dataframe of predictions
df = pd.DataFrame()
# Iterating through individual game prediction tables from 538 and putting into one dataframe
for table in tables[:36]:
    if table.shape[0] == 2:
        # Only getting rows/columns/tables I need
        table = table.iloc[:, [1,3]]
        table.columns = ['team', 'win_prob']
        # Appending to list that will be appended to ultimate df
        data_list = []
        for index, row in table.iterrows():
            data_list.append(row.team)
            data_list.append(row.win_prob)
        df = df.append(pd.Series(data_list), ignore_index = True)
df.columns = ['team_1', 'team_1_prob', 'team_2', 'team_2_prob']

In [3]:
# Getting odds

# Instantiating WebDriver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.actionnetwork.com/nfl/odds')
# Navigating to Moneyline odds
ml_button = driver.find_element_by_xpath("//*[@id='__next']/div/main/div/div[2]/div/div[1]/div[2]/select")
select = Select(ml_button)
select.select_by_visible_text('Moneyline')
# Getting data into pandas and formatting
html = driver.page_source
tables = pd.read_html(html)
odds = tables[0]
odds = odds.iloc[::2]
odds.reset_index(drop = True, inplace = True)
# Getting list of team
teams = list(df.team_1.unique())
teams.extend(list(df.team_2.unique()))
teams = list(set(teams))
# Function to convert odds to probability
def _calculate_odds(odds):
    if odds<0:
        return (abs(odds)/(abs(odds)+100))*100
    if odds>0:
        return (100/(odds+100))*100
# Iterating through to get home/away and odds
odds_df = pd.DataFrame(columns = ['Home_Team', 'Away_Team', 'Home_Odds', 'Away_Odds'])
for index, row in odds.iterrows():
    # Retreiving home and away teams
    teams_dict = {}
    for team in teams:
        if row.Scheduled.find(team) != -1:
            teams_dict[row.Scheduled.find(team)] = team
    keys = []
    for key in teams_dict.keys():
        keys.append(key)
    if keys[0] > keys[1]:
        home_team = teams_dict[keys[0]]
        away_team = teams_dict[keys[1]]
    else:
        home_team = teams_dict[keys[1]]
        away_team = teams_dict[keys[0]]
    # Retreiving odds
    ml_string = row['Unnamed: 5']
    if len(ml_string) == 8:
        ml_away = ml_string[:4]
        ml_home = ml_string[-4:]
    elif len(ml_string) == 9:
        if (ml_string[4] == '+') | (ml_string[4]=='-'):
            ml_away = ml_string[:4]
            ml_home = ml_string[-5:]
        else:
            ml_away = ml_string[:5]
            ml_home = ml_string[-4:]
    elif len(ml_string) == 10:
            ml_away = ml_string[:5]
            ml_home = ml_string[-5:]
    else:
        continue
    try:
        ml_away = float(ml_away)
    except:
        continue
    try:
        ml_home = float(ml_home)
    except:
        continue
    # Appending to df
    series = pd.Series([home_team, away_team, ml_home, ml_away], index = odds_df.columns)
    odds_df = odds_df.append(series, ignore_index = True)
# Creating columns converting odds to probability
odds_df['Home_Prob'] = odds_df.Home_Odds.apply(_calculate_odds)
odds_df['Away_Prob'] = odds_df.Away_Odds.apply(_calculate_odds)



====== WebDriver manager ======
Current google-chrome version is 105.0.5195
Get LATEST driver version for 105.0.5195
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/105.0.5195.52/chromedriver] found in cache


In [11]:
capital = 100000

# Defining KC calculator function
def _calculate_kc(row, kelly, Home):
    if Home:
        diff = row.Home_Prob_538 - row.Home_Prob_Odds
        if diff<0:
            return 0
        else:
            p = row.Home_Prob_538
            q = 1-p
            ml = row.Home_Odds
            if ml>=0:
                b = (ml/100)
            if ml<0:
                b = (100/abs(ml))
            kc = ((p*b) - q) / b
            if (kc > 0.5) & (kc<0.6):
                return kc/(kelly+2)
            if (kc > 0.6) & (kc<0.7):
                return kc/(kelly+4)
            if kc > 0.7:
                return kc/(kelly+7)
            else:
                return kc/kelly
    if not Home:
        diff = row.Away_Prob_538 - row.Away_Prob_Odds
        if diff<0:
            return 0
        else:
            p = row.Away_Prob_538
            q = 1-p
            ml = row.Away_Odds
            if ml>=0:
                b = (ml/100)
            if ml<0:
                b = (100/abs(ml))
            kc = ((p*b) - q) / b
            if (kc > 0.5) & (kc<0.6):
                return kc/(kelly+2)
            if (kc > 0.6) & (kc<0.7):
                return kc/(kelly+4)
            if kc > 0.7:
                return kc/(kelly+7)
            else:
                return kc/kelly

# Joining odds to 538 projections and generating bets

final_df = df.merge(odds_df, left_on = 'team_2', right_on = 'Home_Team')
final_df = final_df[['Home_Team', 'Away_Team', 'Home_Odds', 'Away_Odds', 'Home_Prob', 'Away_Prob', 
                    'team_2_prob', 'team_1_prob']]
final_df.columns = ['Home_Team', 'Away_Team', 'Home_Odds', 'Away_Odds', 'Home_Prob_Odds', 'Away_Prob_Odds', 
                    'Home_Prob_538', 'Away_Prob_538']
# Formatting columns for KC
final_df['Home_Prob_538'] = final_df.Home_Prob_538.str.strip('%')
final_df['Away_Prob_538'] = final_df.Away_Prob_538.str.strip('%')
final_df['Home_Prob_538'] = final_df.Home_Prob_538.astype('float') / 100.0
final_df['Away_Prob_538'] = final_df.Away_Prob_538.astype('float') / 100.0
final_df['Home_Prob_Odds'] = final_df.Home_Prob_Odds / 100.0
final_df['Away_Prob_Odds'] = final_df.Away_Prob_Odds / 100.0
# Creating bets
final_df['Home_KC'] = final_df.apply(_calculate_kc, axis = 1, kelly = 10, Home = True)
final_df['Away_KC'] = final_df.apply(_calculate_kc, axis = 1, kelly = 10, Home = False)
final_df['Bet'] = final_df.apply(lambda x: capital * x.Home_KC if x.Home_KC>0
        else capital * x.Away_KC, axis = 1)

,Home_Team,Away_Team,Home_Odds,Away_Odds,Home_Prob_Odds,Away_Prob_Odds,Home_Prob_538,Away_Prob_538,Home_KC,Away_KC,Bet
0,Rams,Bills,106.0,-124.0,0.485437,0.553571,0.53,0.47,0.008660,0.000000,866.037736
1,Jets,Ravens,240.0,-295.0,0.294118,0.746835,0.38,0.62,0.012167,0.000000,1216.666667
2,Panthers,Browns,-118.0,100.0,0.541284,0.500000,0.57,0.43,0.006260,0.000000,626.000000
3,Texans,Colts,270.0,-335.0,0.270270,0.770115,0.33,0.67,0.008185,0.000000,818.518519
4,Commanders,Jaguars,-154.0,130.0,0.606299,0.434783,0.68,0.32,0.018720,0.000000,1872.000000
5,Dolphins,Patriots,-168.0,142.0,0.626866,0.413223,0.58,0.42,0.000000,0.001155,115.492958
6,Falcons,Saints,210.0,-255.0,0.322581,0.718310,0.36,0.64,0.005524,0.000000,552.380952
7,Lions,Eagles,210.0,-255.0,0.322581,0.718310,0.40,0.60,0.011429,0.000000,1142.857143
8,Bengals,Steelers,-330.0,265.0,0.767442,0.273973,0.68,0.32,0.000000,0.006340,633.962264
9,Bears,49ers,215.0,-260.0,0.317460,0.722222,0.37,0.63,0.007698,0.000000,769.767442


In [4]:
odds_df

,Home_Team,Away_Team,Home_Odds,Away_Odds,Home_Prob,Away_Prob
0,Seahawks,Broncos,220.0,-270.0,31.250000,72.972973
1,Rams,Bills,106.0,-124.0,48.543689,55.357143
2,Lions,Eagles,210.0,-255.0,32.258065,71.830986
3,Falcons,Saints,210.0,-255.0,32.258065,71.830986
4,Jets,Ravens,240.0,-295.0,29.411765,74.683544
5,Bengals,Steelers,-330.0,265.0,76.744186,27.397260
6,Dolphins,Patriots,-168.0,142.0,62.686567,41.322314
7,Commanders,Jaguars,-154.0,130.0,60.629921,43.478261
8,Bears,49ers,215.0,-260.0,31.746032,72.222222
9,Panthers,Browns,-118.0,100.0,54.128440,50.000000


In [5]:
df

,team_1,team_1_prob,team_2,team_2_prob
0,Bills,47%,Rams,53%
1,Ravens,62%,Jets,38%
2,Browns,43%,Panthers,57%
3,Colts,67%,Texans,33%
4,Jaguars,32%,Commanders,68%
5,Patriots,42%,Dolphins,58%
6,Saints,64%,Falcons,36%
7,Eagles,60%,Lions,40%
8,Steelers,32%,Bengals,68%
9,49ers,63%,Bears,37%


In [71]:
# Getting list of team

teams = list(df.team_1.unique())
teams.extend(list(df.team_2.unique()))
teams = list(set(teams))

32

In [79]:
# Function to get beginning of the year win total projections

tables = pd.read_html(f'https://projects.fivethirtyeight.com/{dt.date.today().year}-nfl-predictions/?ex_cid=rrpromo')
df = tables[0]
# Getting rid of multi-index
df.columns = df.columns.droplevel(0)

In [86]:
df = df[['team', 'recordsim. record', 'point diff.', 'make playoffs', 'win division']]
df.columns = ['team', 'proj_record', 'proj_points_diff', 'prob_playoffs', 'prob_division']
df.to_csv('BOY_projections.csv')